In [1]:
import pandas as pd
from collections import defaultdict
from datasketch import MinHashLSHForest, MinHash

In [2]:
def jaccard_similarity(A, B):
    # Convert the sets to sets for better performance
    set_A = set(A)
    set_B = set(B)

    # Calculate the intersection size
    intersection = len(set_A & set_B)

    # Calculate the union size
    union = len(set_A | set_B)

    # Take the ratio of sizes
    if union == 0:
        return 0
    similarity = intersection / union

    return similarity

In [3]:
df = pd.read_csv('tweets_2022_abril_junio.csv') # cuidado al ejecutar

In [4]:
df.drop(['id', 'created_at', 'retweet_count', 'favorite_count'], axis=1, inplace=True)

In [5]:
df.shape

(4594980, 2)

In [6]:
tweets_originales = df.loc[df['text'].str[:4].str.contains('RT @',case=False) == False]

rt = df.loc[df['text'].str[:4].str.contains('RT @',case=False) == True]

In [7]:
tweets_originales.head()

,screen_name,text
6,simonlodijo,@unveranonaranja @ruidosafest @franciscamusic ...
9,MacaSimplemente,@LaSuRivas @ElisaLoncon @siliconvalle @Valdebe...
12,LuisVer75699645,@teanval0207 @izkia @arturozunigaj Excelente...
13,MITERRED,@mcubillossigall https://t.co/gkg5PwbZhZ
14,piaelizabethvm,@simonlodijo @ruidosafest @franciscamusic @gio...


In [8]:
rt.head()

,screen_name,text
0,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...
1,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...
2,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...
3,rosmarieher,RT @melnicksergio: la pelotudez no tiene limit...
4,GQuelluen,RT @BSepulvedaHales: Ante la circulación de no...


In [9]:
retweets = pd.DataFrame()

split_text = rt['text'].str.split(' ', expand=True, n=2)
retweets['user'] = split_text[1].str.strip("@").str.strip(':')
retweets['tweet'] = split_text[2]

retweets.reset_index(drop=True, inplace=True)

tweets_originales.rename(columns={'screen_name': 'user', 'text': 'tweet'}, inplace=True)

tweets = pd.concat([tweets_originales, retweets]).reset_index(drop=True)

final_tweets = tweets.drop_duplicates(['user', 'tweet'], ignore_index=True)

final_tweets

C:\Users\jor_x\AppData\Local\Temp\ipykernel_23952\4236830760.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_originales.rename(columns={'screen_name': 'user', 'text': 'tweet'}, inplace=True)


,user,tweet
0,simonlodijo,@unveranonaranja @ruidosafest @franciscamusic ...
1,MacaSimplemente,@LaSuRivas @ElisaLoncon @siliconvalle @Valdebe...
2,LuisVer75699645,@teanval0207 @izkia @arturozunigaj Excelente...
3,MITERRED,@mcubillossigall https://t.co/gkg5PwbZhZ
4,piaelizabethvm,@simonlodijo @ruidosafest @franciscamusic @gio...
...,...,...
1442712,hocikonapatriot,@tere_marinovic fuiste la voz de muchos chilen...
1442713,Caroline_walls6,@tere_marinovic Si la Tere pudiera decir lo qu...
1442714,EduardoAlegri16,@tere_marinovic Justamente la mayor ausente e...
1442715,AlonsoRPatriota,"@maluchayallego Historica debería ser, el dar ..."


In [10]:
tweet_shingles = {}

k = 4
batch_size = 100000 # La cantidad del lote depende unicamente de la capacidad el hardware donde se ejecuta
cantidad = final_tweets.shape[0]

for carga in range(0, cantidad, batch_size):
    batch_end = min(carga + batch_size, cantidad)
    batch_tweets = final_tweets.iloc[carga:batch_end]

    for tweet in batch_tweets.itertuples():
        user = tweet.user
        tweet_text = tweet.tweet

        shingles = set(tweet_text[i:i+k] for i in range(len(tweet_text) - k + 1))
        tweet_shingles.setdefault(user, set()).update(shingles)

In [12]:
# Crear el índice LSH
forest = MinHashLSHForest(num_perm=128)  # Ajusta el número de permutaciones según tus necesidades

# Agregar los MinHash de los usuarios al índice LSH
for user, shingles in tweet_shingles.items():
    minhash = MinHash(num_perm=128)  # Ajusta el número de permutaciones según tus necesidades
    
    for shingle in shingles:
        minhash.update(shingle.encode('utf-8'))
    
    forest.add(user, minhash)

# Construir el índice LSH
forest.index()

# Definir el umbral de similaridad de Jaccard
threshold = 0.6

# Encontrar usuarios similares utilizando LSH
user_similares = defaultdict(set)

for user, shingles in tweet_shingles.items():
    minhash_query = MinHash(num_perm=128)  # Ajusta el número de permutaciones según tus necesidades
    
    for shingle in shingles:
        minhash_query.update(shingle.encode('utf-8'))
    
    # Consultar el índice LSH para encontrar los usuarios similares
    result = forest.query(minhash_query, k=4)  # Ajusta el valor de 'k' según tus necesidades
    
    for similar_user in result:
        if similar_user != user:  # Excluir el mismo usuario de la lista de usuarios similares
            similarity = jaccard_similarity(tweet_shingles[user], tweet_shingles[similar_user])
            if threshold <= similarity < 0.9:
                user_similares[user].add((similar_user, similarity))

In [13]:
filtered_user = {x: user_similares[x] for x in user_similares.keys() if len(final_tweets.loc[final_tweets['user'] == x]) >= 3}

In [14]:
filtered_user

{'Ilustradocl': {('andystefano', 0.7586206896551724)},
 'Luisgermanleon': {('GatinesG', 0.6666666666666666)},
 'OmRadioAr': {('OmNoticiasAr', 0.6923076923076923)},
 'OmNoticiasAr': {('OmRadioAr', 0.6923076923076923)},
 'JuanaVergaraMu1': {('Marceloalexia1', 0.6216216216216216)},
 'gonzaloignacioh': {('polatapia', 0.6666666666666666)},
 'Jaimefuenteal10': {('f57271a3ae2c433', 0.7352941176470589),
  ('gonzalotv13', 0.7058823529411765),
  ('mfaziom', 0.6857142857142857)},
 'jminnovando': {('jmovandojeria', 0.6273164035689773)},
 'Red_nuble': {('Red_OHiggins', 0.6302816901408451)},
 'Rodrigo69982619': {('Luz_Carimoney', 0.6229508196721312)},
 'salvador_vimua': {('Paonessa', 0.6666666666666666),
  ('Weedo04060798', 0.6470588235294118)},
 'andystefano': {('Ilustradocl', 0.7586206896551724)},
 'ladyvichile': {('kirtclem', 0.6956521739130435)},
 'Fco__Vasquez': {('Hermini63800027', 0.7391304347826086),
  ('SportZeta', 0.7083333333333334),
  ('erikabriones', 0.7391304347826086)},
 'eolicochile'

In [32]:
final_tweets.loc[final_tweets['user'] == 'Luisgermanleon']

,user,tweet
37300,Luisgermanleon,#Valdiviacl
227315,Luisgermanleon,#ValdiviaCL
418266,Luisgermanleon,#valdiviacl


In [28]:
filtered_user['Ilustradocl']

{('andystefano', 0.7586206896551724)}

In [29]:
filtered_user['andystefano']

{('Ilustradocl', 0.7586206896551724)}

In [16]:
final_tweets.loc[final_tweets['user'] == 'andystefano']

,user,tweet
111101,andystefano,La ironía de Jorge Baradit tras llamativo disc...
128802,andystefano,Desde “pervertido” a “escombro”: la fuerte pel...
149418,andystefano,Madre de Boric: “Gabriel se inmoló por la Conv...
417647,andystefano,Estudio indica que 58% de las personas ha reci...
542181,andystefano,Javier Olivares indignado con la Convención Co...
824064,andystefano,Teresa Marinovic confirma que donará todo el d...
906008,andystefano,El nuevo pataleo de Teresa Marinovic: rechazó ...
1029804,andystefano,Rayan monumento de Huanchaca en contra de la C...
1220574,andystefano,Guarello dice que Baradit cobró por dar charla...
1232527,andystefano,Ministra Jara y presidenta de la Convención Co...


In [23]:
final_tweets.loc[final_tweets['user'] == 'Ilustradocl']

,user,tweet
12419,Ilustradocl,Desde “pervertido” a “escombro”: la fuerte pel...
53107,Ilustradocl,La ironía de Jorge Baradit tras llamativo disc...
208428,Ilustradocl,Madre de Boric: “Gabriel se inmoló por la Conv...
249197,Ilustradocl,Estudio indica que 58% de las personas ha reci...
487798,Ilustradocl,Javier Olivares indignado con la Convención Co...
728091,Ilustradocl,Teresa Marinovic confirma que donará todo el d...
754377,Ilustradocl,Ministra Jara y presidenta de la Convención Co...
851438,Ilustradocl,El nuevo pataleo de Teresa Marinovic: rechazó ...
947627,Ilustradocl,Guarello dice que Baradit cobró por dar charla...
1070729,Ilustradocl,Rayan monumento de Huanchaca en contra de la C...
